In [30]:
import numpy as np
from functools import reduce
import operator

import string
from typing import Dict, List, Optional, Tuple, Union

import json
import openai
from openai import OpenAI

system_prompt_template = string.Template("""You are an evolutionary algorithm assistant responsible for generating a new mean genome based on the results of previous generations to create parameter candidates.

When you respond, please output a JSON where:
1. "thought": This key represents your reasoning process when designing the mean genome for the next generation, including insights from the selected top-performing genomes.
2. "mean_genome": This key provides a list of integer values between 0 and 100, consistent with the specified genomic dimensions. This mean genome is used for mutations in the next generation.
Ensure that the "next_mean_genome" generation aligns with the "thought" analysis provided.
Here is an example:
{
  "thought": "Among the top-performing genomes, Genomes 1, 7, and 9 showed high fitness...",
  "mean_genome": [345, 15, 838, ...]
}

Ensure that the mean genome has dimensions $dimensions and improves upon the highest fitness score from the previous generation by optimizing parameter combinations.

You will be provided with the previous generation's genomes and their fitness scores. 
""")

user_prompt_template = string.Template("""Generation $generation:

The highest fitness score:
$prev_best_cost

Top $top_k Genomes from Previous Generation (sorted by fitness score):
$prev_generation

Using these top $top_k genomes, please provide the next mean genome based on their values and performance insights.
""")


# LLMEvolutionStrategy
class LLMEvolutionStrategy:
    def __init__(self, api_key, model, system_prompt_template, user_prompt_template, dimensions, population_size, top_k):
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.dimensions = dimensions  
        self.population_size = population_size
        self.system_prompt = system_prompt_template.substitute(dimensions=dimensions)
        self.user_prompt_template = user_prompt_template
        self.messages = [{"role": "system", "content": self.system_prompt}]
        self.thought = ''
        self.prev_best_cost = -np.inf
        self.prev_best_genome = None
        self.generation = 0
        self.top_k = top_k
        
    def mutate(self, pre_gen: np.array, fit_scores: List[float], max_retries: int = 3, sigma_low:float=0.05, sigma_high:float=0.2) -> np.array:
        self.generation += 1
        fitness_improved = max(fit_scores) > self.prev_best_cost
        
        # Scale and discretize pre_gen to integer-based format for LLM compatibility
        pre_gen_list=[]
        for idx, (gen, score) in enumerate(zip(pre_gen, fit_scores)):
            if score > self.prev_best_cost:
                self.prev_best_cost = score
                self.prev_best_genome = gen
            gen = (gen * 100).astype(int).reshape(*self.dimensions).tolist()
            pre_gen_list.append({"fitness_score": score, f"genome_{idx+1}": gen})
            
        pre_gen_list = sorted(pre_gen_list, key=lambda x: x['fitness_score'], reverse=True)[:self.top_k]
        user_prompt = self.user_prompt_template.substitute(
            generation=self.generation,
            prev_generation=pre_gen_list,
            prev_best_cost=self.prev_best_cost,
            top_k=self.top_k,
        )
        self.messages.append({"role": "user", "content": user_prompt})

        json_output = self._get_response_with_validation(max_retries)
        self.thought = json_output['thought']

        # Convert the mean genome back to [0, 1] range
        mean_genome = np.array(json_output['mean_genome']) / 100

        # Generate the new population by applying Gaussian mutation to the mean genome
        new_generation = []
        # Apply Gaussian noise with varying sigma depending on fitness improvement
        sigma = sigma_low if fitness_improved else sigma_high
        for _ in range(self.population_size):
            mutated_genome = self.gaussian_mutation(mean_genome, sigma)
            new_generation.append(mutated_genome.flatten())
        return np.stack(new_generation, axis=0)

    def gaussian_mutation(self, genome, sigma):
        mutation = np.random.normal(0, sigma, size=genome.shape)
        mutated_genome = genome + mutation
        return np.clip(mutated_genome, 0, 1)  # Keep values within [0, 1]

    def _get_response_with_validation(self, max_retries):
        retry_count = 0
        while retry_count < max_retries:
            try:
                completion = self.get_completion(self.messages)
                self.messages.append(completion)
                json_output = json.loads(completion['content'])
                dimension_check = self.genome_dimension_check(json_output)
                if dimension_check is True:
                    return json_output
                else:
                    error_message = dimension_check
                    print("GENERATION NOT VALID")
                    self.messages.append({
						"role": "user",
						"content": f"Genome dimensions not valid. Error:\n{error_message}\nPlease regenerate the next mean genome."
					})
                    retry_count += 1
            except openai.error.AuthenticationError as e:
                print(f"Authentication error: {e}")
                raise e  
            except (openai.error.Timeout, openai.error.RateLimitError, openai.error.APIError) as e:
                print(f"OpenAI API error: {e}")
                retry_count += 1
                continue

        raise ValueError("Maximum retries reached. Unable to get a valid response.")

    def genome_dimension_check(self, json_output: dict) -> str:
        genome_array = np.array(json_output['mean_genome'])
        if genome_array.shape == self.dimensions:
            return True
        else:
            return (f"The mean_genome's dimensions {genome_array.shape} do not match the specified dimensions {self.dimensions}.")
                                          
    def get_completion(self, messages):
        return self.client.chat.completions.create(
            model=self.model,
            n=1,
            messages=messages,
            temperature=0,
            response_format={"type": "json_object"},
        ).choices[0].message.to_dict()


n_params=8
api_key='API-KEY'
llm_model='gpt-4o-mini'

dimensions=(n_params,)
population_size = 4 + round(3*np.log(reduce(operator.mul, dimensions, 1)))
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model=llm_model,
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

In [59]:
import numpy as np
import torch

def sphere_function(x):
    return sum([xi**2 for xi in x])

def rosenbrock_function(x):
    return sum([100*(x[i+1] - x[i]**2)**2 + (x[i] - 1)**2 for i in range(len(x)-1)])

def rastrigin_function(x):
    return 10 * len(x) + sum([(xi**2 - 10 * np.cos(2 * np.pi * xi)) for xi in x])

def ackley_function(x):
    part1 = -20 * np.exp(-0.2 * np.sqrt(sum([xi**2 for xi in x]) / len(x)))
    part2 = -np.exp(sum([np.cos(2 * np.pi * xi) for xi in x]) / len(x))
    return part1 + part2 + 20 + np.e

def griewank_function(x):
    part1 = sum([xi**2 for xi in x]) / 4000
    part2 = np.prod([np.cos(xi / np.sqrt(i+1)) for i, xi in enumerate(x)])
    return part1 - part2 + 1

def schwefel_function(x):
    return 418.9829 * len(x) - sum([xi * np.sin(np.sqrt(abs(xi))) for xi in x])

## sphere_function

In [58]:
import numpy as np
from cma import CMAEvolutionStrategy

def sphere_function(x):
    return sum([xi**2 for xi in x])

# Set up the CMA-ES optimizer with an 8-dimensional input and initial guess
dimension = 8
initial_guess = np.random.randn(dimension)  # random initial guess
sigma = 1/6

# Instantiate the CMA-ES optimizer
es = CMAEvolutionStrategy(initial_guess, sigma)

# Optimize the Rosenbrock function using CMA-ES
max_iterations = 30  # Maximum number of iterations
for generation in range(max_iterations):
    solutions = es.ask()  # Get new candidate solutions
    fitnesses = [sphere_function(sol) for sol in solutions]  # Evaluate solutions
    es.tell(solutions, fitnesses)  # Provide fitness values to CMA-ES

    # Print the progress every 100 generations
    if generation % 1 == 0:
        print(f"Generation {generation}: Best fitness = {es.result.fbest}")

    # Check for convergence or stop after a set number of iterations
    if es.stop():
        break

# Print the final result
print("Best solution found: ", es.result.xbest)
print("Best fitness achieved: ", es.result.fbest)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=501861, Sun Oct 20 15:53:06 2024)
Generation 0: Best fitness = 4.693488982299108
Generation 1: Best fitness = 3.748651954537272
Generation 2: Best fitness = 3.6320887922800082
Generation 3: Best fitness = 3.182525308172284
Generation 4: Best fitness = 1.9836579926917384
Generation 5: Best fitness = 1.1856095101630086
Generation 6: Best fitness = 1.1856095101630086
Generation 7: Best fitness = 0.8630665869633651
Generation 8: Best fitness = 0.8630665869633651
Generation 9: Best fitness = 0.8630665869633651
Generation 10: Best fitness = 0.7766155496966725
Generation 11: Best fitness = 0.4570529754884377
Generation 12: Best fitness = 0.4570529754884377
Generation 13: Best fitness = 0.4570529754884377
Generation 14: Best fitness = 0.4570529754884377
Generation 15: Best fitness = 0.3159281208099427
Generation 16: Best fitness = 0.3095083241473614
Generation 17: Best fitness = 0.2860992364405588
Generation 18: Best fitness = 0.28609923

In [60]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model=llm_model,
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-sphere_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-sphere_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 1.1492396695931515
Generation 1: Best fitness = 1.205040882765584
Generation 2: Best fitness = 0.4894664971337016
Generation 3: Best fitness = 0.693169674712264
Generation 4: Best fitness = 0.7383526294077123
Generation 5: Best fitness = 0.3419786711244128
Generation 6: Best fitness = 0.26136023972512445
Generation 7: Best fitness = 0.2244365832565567
Generation 8: Best fitness = 0.24610903846765686
Generation 9: Best fitness = 0.04668554078154721
Generation 10: Best fitness = 0.07046483274670756
Generation 11: Best fitness = 0.11704822351149512
Generation 12: Best fitness = 0.13155263435323647
Generation 13: Best fitness = 0.021158428415053766
Generation 14: Best fitness = 0.023740819240786824
Generation 15: Best fitness = 0.047467487804468564
Generation 16: Best fitness = 0.06243265994594639
Generation 17: Best fitness = 0.014642604346367688
Generation 18: Best fitness = 0.03834195317326776
Generation 19: Best fitness = 0.045031126056360046
Generation 20:

## rosenbrock_function

In [69]:
import numpy as np
from cma import CMAEvolutionStrategy

# Set up the CMA-ES optimizer with an 8-dimensional input and initial guess
dimension = 8
initial_guess = np.random.randn(dimension)  # random initial guess
sigma = 1/6

# Instantiate the CMA-ES optimizer
es = CMAEvolutionStrategy(initial_guess, sigma)

# Optimize the Rosenbrock function using CMA-ES
max_iterations = 30  # Maximum number of iterations
for generation in range(max_iterations):
    solutions = es.ask()  # Get new candidate solutions
    fitnesses = [rosenbrock_function(sol) for sol in solutions]  # Evaluate solutions
    es.tell(solutions, fitnesses)  # Provide fitness values to CMA-ES

    # Print the progress every 100 generations
    if generation % 1 == 0:
        print(f"Generation {generation}: Best fitness = {es.result.fbest}")

    # Check for convergence or stop after a set number of iterations
    if es.stop():
        break

# Print the final result
print("Best solution found: ", es.result.xbest)
print("Best fitness achieved: ", es.result.fbest)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=523835, Sun Oct 20 16:04:24 2024)
Generation 0: Best fitness = 1050.0484716468382
Generation 1: Best fitness = 768.5011831059167
Generation 2: Best fitness = 478.1373362563849
Generation 3: Best fitness = 351.2781869776947
Generation 4: Best fitness = 178.4912659401508
Generation 5: Best fitness = 161.71740485111542
Generation 6: Best fitness = 136.54820181677218
Generation 7: Best fitness = 47.92056174495844
Generation 8: Best fitness = 39.959548210365334
Generation 9: Best fitness = 39.959548210365334
Generation 10: Best fitness = 39.959548210365334
Generation 11: Best fitness = 39.959548210365334
Generation 12: Best fitness = 39.959548210365334
Generation 13: Best fitness = 39.959548210365334
Generation 14: Best fitness = 34.42941828947585
Generation 15: Best fitness = 26.701027789031954
Generation 16: Best fitness = 26.701027789031954
Generation 17: Best fitness = 26.701027789031954
Generation 18: Best fitness = 26.7010277890

In [65]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model=llm_model,
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-rosenbrock_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-rosenbrock_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 48.28541025547237
Generation 1: Best fitness = 48.25319622983326
Generation 2: Best fitness = 42.79452823847926
Generation 3: Best fitness = 39.095285656827116
Generation 4: Best fitness = 37.35522656742984
Generation 5: Best fitness = 34.93021683858691
Generation 6: Best fitness = 28.337338600137006
Generation 7: Best fitness = 29.085744567418615
Generation 8: Best fitness = 32.594698465285056
Generation 9: Best fitness = 37.88572939669892
Generation 10: Best fitness = 41.24417509980272
Generation 11: Best fitness = 33.40630247047736
Generation 12: Best fitness = 38.854471858487216
Generation 13: Best fitness = 42.99059291826699
Generation 14: Best fitness = 44.59041221001695
Generation 15: Best fitness = 42.198554981653544
Generation 16: Best fitness = 36.91231894517312
Generation 17: Best fitness = 48.36809359990532
Generation 18: Best fitness = 51.502775310115545
Generation 19: Best fitness = 37.359362746405324
Generation 20: Best fitness = 38.745720037

## rastrigin_function

In [68]:
import numpy as np
from cma import CMAEvolutionStrategy

# Set up the CMA-ES optimizer with an 8-dimensional input and initial guess
dimension = 8
initial_guess = np.random.randn(dimension)  # random initial guess
sigma = 1/6

# Instantiate the CMA-ES optimizer
es = CMAEvolutionStrategy(initial_guess, sigma)

# Optimize the Rosenbrock function using CMA-ES
max_iterations = 30  # Maximum number of iterations
for generation in range(max_iterations):
    solutions = es.ask()  # Get new candidate solutions
    fitnesses = [rastrigin_function(sol) for sol in solutions]  # Evaluate solutions
    es.tell(solutions, fitnesses)  # Provide fitness values to CMA-ES

    # Print the progress every 100 generations
    if generation % 1 == 0:
        print(f"Generation {generation}: Best fitness = {es.result.fbest}")

    # Check for convergence or stop after a set number of iterations
    if es.stop():
        break

# Print the final result
print("Best solution found: ", es.result.xbest)
print("Best fitness achieved: ", es.result.fbest)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=425307, Sun Oct 20 16:04:03 2024)
Generation 0: Best fitness = 68.80582574375156
Generation 1: Best fitness = 61.26410969652052
Generation 2: Best fitness = 46.08585094387182
Generation 3: Best fitness = 35.179069221351725
Generation 4: Best fitness = 31.65686184837886
Generation 5: Best fitness = 31.65686184837886
Generation 6: Best fitness = 31.65686184837886
Generation 7: Best fitness = 31.53669494487861
Generation 8: Best fitness = 31.53669494487861
Generation 9: Best fitness = 22.307737928279643
Generation 10: Best fitness = 22.307737928279643
Generation 11: Best fitness = 21.96590702070373
Generation 12: Best fitness = 20.749534214929362
Generation 13: Best fitness = 20.749534214929362
Generation 14: Best fitness = 20.749534214929362
Generation 15: Best fitness = 18.133310135216526
Generation 16: Best fitness = 18.022646720249924
Generation 17: Best fitness = 18.022646720249924
Generation 18: Best fitness = 17.7284786918591

In [72]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model=llm_model,
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-rastrigin_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-rastrigin_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 119.93354142917428
Generation 1: Best fitness = 83.01660610016644
Generation 2: Best fitness = 51.96970720666159
Generation 3: Best fitness = 68.41130818711508
Generation 4: Best fitness = 47.315445409282056
Generation 5: Best fitness = 46.67121656431195
Generation 6: Best fitness = 45.80468613299563
Generation 7: Best fitness = 39.87317236785695
Generation 8: Best fitness = 36.42566004685576
Generation 9: Best fitness = 28.604158244309076
Generation 10: Best fitness = 26.31797480189544
Generation 11: Best fitness = 22.516522924066052
Generation 12: Best fitness = 20.941695982156894
Generation 13: Best fitness = 19.68361545506815
Generation 14: Best fitness = 15.392801652651073
Generation 15: Best fitness = 10.86324194447127
Generation 16: Best fitness = 6.042191153920669
Generation 17: Best fitness = 7.322726436500417
Generation 18: Best fitness = 12.074792084654831
Generation 19: Best fitness = 5.410163849476049
Generation 20: Best fitness = 10.9770707894

In [99]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model='gpt-4o',
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-rastrigin_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-rastrigin_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 106.46564369445593
Generation 1: Best fitness = 56.59265374536088
Generation 2: Best fitness = 65.36959432884645
Generation 3: Best fitness = 42.29130128940946
Generation 4: Best fitness = 41.97668241559688
Generation 5: Best fitness = 37.72481358166026
Generation 6: Best fitness = 35.016934225946045
Generation 7: Best fitness = 32.735090529983964
Generation 8: Best fitness = 29.36045590922501
Generation 9: Best fitness = 27.914903244128766
Generation 10: Best fitness = 23.928028423145733
Generation 11: Best fitness = 20.720120155349143
Generation 12: Best fitness = 15.575553971359966
Generation 13: Best fitness = 12.179228650416874
Generation 14: Best fitness = 10.823616782693676
Generation 15: Best fitness = 11.161943988447987
Generation 16: Best fitness = 13.181599578164551
Generation 17: Best fitness = 16.725283579688792
Generation 18: Best fitness = 24.581316937784862
Generation 19: Best fitness = 20.921177938488263
Generation 20: Best fitness = 25.962

## ackley_function

In [67]:
import numpy as np
from cma import CMAEvolutionStrategy

# Set up the CMA-ES optimizer with an 8-dimensional input and initial guess
dimension = 8
initial_guess = np.random.randn(dimension)  # random initial guess
sigma = 1/6

# Instantiate the CMA-ES optimizer
es = CMAEvolutionStrategy(initial_guess, sigma)

# Optimize the Rosenbrock function using CMA-ES
max_iterations = 30  # Maximum number of iterations
for generation in range(max_iterations):
    solutions = es.ask()  # Get new candidate solutions
    fitnesses = [ackley_function(sol) for sol in solutions]  # Evaluate solutions
    es.tell(solutions, fitnesses)  # Provide fitness values to CMA-ES

    # Print the progress every 100 generations
    if generation % 1 == 0:
        print(f"Generation {generation}: Best fitness = {es.result.fbest}")

    # Check for convergence or stop after a set number of iterations
    if es.stop():
        break

# Print the final result
print("Best solution found: ", es.result.xbest)
print("Best fitness achieved: ", es.result.fbest)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=532360, Sun Oct 20 16:03:51 2024)
Generation 0: Best fitness = 5.6124446678886315
Generation 1: Best fitness = 5.211250452194065
Generation 2: Best fitness = 5.211250452194065
Generation 3: Best fitness = 4.96129928895704
Generation 4: Best fitness = 4.96129928895704
Generation 5: Best fitness = 4.96129928895704
Generation 6: Best fitness = 4.8791169597728725
Generation 7: Best fitness = 4.8791169597728725
Generation 8: Best fitness = 4.8791169597728725
Generation 9: Best fitness = 4.8791169597728725
Generation 10: Best fitness = 4.8791169597728725
Generation 11: Best fitness = 4.785329369766256
Generation 12: Best fitness = 4.761763532190933
Generation 13: Best fitness = 4.616903935091798
Generation 14: Best fitness = 4.16312614394381
Generation 15: Best fitness = 3.965106606710798
Generation 16: Best fitness = 3.9271704283449407
Generation 17: Best fitness = 3.7529856311386536
Generation 18: Best fitness = 3.7529856311386536
Ge

In [73]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model=llm_model,
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-ackley_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-ackley_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 4.219356634393151
Generation 1: Best fitness = 3.94226880062206
Generation 2: Best fitness = 3.8312617281727666
Generation 3: Best fitness = 3.513122700260237
Generation 4: Best fitness = 3.9312473574582403
Generation 5: Best fitness = 3.645729897506517
Generation 6: Best fitness = 3.7306490278958404
Generation 7: Best fitness = 3.679699233756143
Generation 8: Best fitness = 3.432044650725754
Generation 9: Best fitness = 3.765709458697256
Generation 10: Best fitness = 3.3467680612297888
Generation 11: Best fitness = 3.5409032416991306
Generation 12: Best fitness = 3.2727028414686186
Generation 13: Best fitness = 3.5580353089850836
Generation 14: Best fitness = 3.364312219777752
Generation 15: Best fitness = 2.847391871938101
Generation 16: Best fitness = 3.1962757916144864
Generation 17: Best fitness = 2.999007630973541
Generation 18: Best fitness = 2.8571951783626344
Generation 19: Best fitness = 3.108616905904388
Generation 20: Best fitness = 3.0809731277

## griewank_function

In [78]:
import numpy as np
from cma import CMAEvolutionStrategy

# Set up the CMA-ES optimizer with an 8-dimensional input and initial guess
dimension = 8
initial_guess = np.random.randn(dimension)  # random initial guess
sigma = 1/6

# Instantiate the CMA-ES optimizer
es = CMAEvolutionStrategy(initial_guess, sigma)

# Optimize the Rosenbrock function using CMA-ES
max_iterations = 30  # Maximum number of iterations
for generation in range(max_iterations):
    solutions = es.ask()  # Get new candidate solutions
    fitnesses = [griewank_function(sol) for sol in solutions]  # Evaluate solutions
    es.tell(solutions, fitnesses)  # Provide fitness values to CMA-ES

    # Print the progress every 100 generations
    if generation % 1 == 0:
        print(f"Generation {generation}: Best fitness = {es.result.fbest}")

    # Check for convergence or stop after a set number of iterations
    if es.stop():
        break

# Print the final result
print("Best solution found: ", es.result.xbest)
print("Best fitness achieved: ", es.result.fbest)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=539592, Sun Oct 20 16:11:18 2024)
Generation 0: Best fitness = 0.7199951662214865
Generation 1: Best fitness = 0.6190666765397388
Generation 2: Best fitness = 0.5657905525887157
Generation 3: Best fitness = 0.504648395518478
Generation 4: Best fitness = 0.42078110484301023
Generation 5: Best fitness = 0.33433889502646574
Generation 6: Best fitness = 0.2721134699043062
Generation 7: Best fitness = 0.23072073863273934
Generation 8: Best fitness = 0.1725780306849497
Generation 9: Best fitness = 0.10321996632213692
Generation 10: Best fitness = 0.10321996632213692
Generation 11: Best fitness = 0.10321996632213692
Generation 12: Best fitness = 0.09862136327539672
Generation 13: Best fitness = 0.09862136327539672
Generation 14: Best fitness = 0.09862136327539672
Generation 15: Best fitness = 0.0922952717310116
Generation 16: Best fitness = 0.07336900646441558
Generation 17: Best fitness = 0.042646300658784164
Generation 18: Best fitnes

In [84]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model=llm_model,
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-griewank_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-griewank_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 0.18528850805476127
Generation 1: Best fitness = 0.1622109339758817
Generation 2: Best fitness = 0.12781526719064107
Generation 3: Best fitness = 0.11634935076333541
Generation 4: Best fitness = 0.09734433460470959
Generation 5: Best fitness = 0.07551685993171209
Generation 6: Best fitness = 0.08131280245305994
Generation 7: Best fitness = 0.06948537898252916
Generation 8: Best fitness = 0.061955573622446414
Generation 9: Best fitness = 0.05839754530116481
Generation 10: Best fitness = 0.050848014887504345
Generation 11: Best fitness = 0.032899366219150794
Generation 12: Best fitness = 0.03615018051033947
Generation 13: Best fitness = 0.023816360342211795
Generation 14: Best fitness = 0.017285162469854343
Generation 15: Best fitness = 0.0075822298123006115
Generation 16: Best fitness = 0.00863244521599138
Generation 17: Best fitness = 0.0033915552000829408
Generation 18: Best fitness = 0.006327576397044066
Generation 19: Best fitness = 0.028534705325502507


## schwefel_function

In [94]:
import numpy as np
from cma import CMAEvolutionStrategy

# Define the Rosenbrock function
def schwefel_function(x):
    return sum([100*(x[i+1] - x[i]**2)**2 + (x[i] - 1)**2 for i in range(len(x)-1)])

# Set up the CMA-ES optimizer with an 8-dimensional input and initial guess
dimension = 8
initial_guess = np.random.randn(dimension)  # random initial guess
sigma = 1/6

# Instantiate the CMA-ES optimizer
es = CMAEvolutionStrategy(initial_guess, sigma)

# Optimize the Rosenbrock function using CMA-ES
max_iterations = 30  # Maximum number of iterations
for generation in range(max_iterations):
    solutions = es.ask()  # Get new candidate solutions
    fitnesses = [schwefel_function(sol) for sol in solutions]  # Evaluate solutions
    es.tell(solutions, fitnesses)  # Provide fitness values to CMA-ES

    # Print the progress every 100 generations
    if generation % 1 == 0:
        print(f"Generation {generation}: Best fitness = {es.result.fbest}")

    # Check for convergence or stop after a set number of iterations
    if es.stop():
        break

# Print the final result
print("Best solution found: ", es.result.xbest)
print("Best fitness achieved: ", es.result.fbest)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=441393, Sun Oct 20 16:26:09 2024)
Generation 0: Best fitness = 3298.8408393101267
Generation 1: Best fitness = 2470.955557784327
Generation 2: Best fitness = 2191.189619092136
Generation 3: Best fitness = 2067.816722747908
Generation 4: Best fitness = 1020.6706460471721
Generation 5: Best fitness = 1020.6706460471721
Generation 6: Best fitness = 541.5565843704014
Generation 7: Best fitness = 228.69255254562066
Generation 8: Best fitness = 189.37963297214776
Generation 9: Best fitness = 189.37963297214776
Generation 10: Best fitness = 181.74575634668187
Generation 11: Best fitness = 181.74575634668187
Generation 12: Best fitness = 181.74575634668187
Generation 13: Best fitness = 74.32298141552407
Generation 14: Best fitness = 74.32298141552407
Generation 15: Best fitness = 70.8957335493063
Generation 16: Best fitness = 56.91961983391385
Generation 17: Best fitness = 56.91961983391385
Generation 18: Best fitness = 56.91961983391385

In [86]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model=llm_model,
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-schwefel_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-schwefel_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 116.49893730053098
Generation 1: Best fitness = 112.76383050749162
Generation 2: Best fitness = 95.19238291574993
Generation 3: Best fitness = 91.58553383232102
Generation 4: Best fitness = 98.71822614199742
Generation 5: Best fitness = 92.9076864243084
Generation 6: Best fitness = 81.93426891207264
Generation 7: Best fitness = 88.3011858008431
Generation 8: Best fitness = 84.89976135662603
Generation 9: Best fitness = 82.75275785973469
Generation 10: Best fitness = 82.85492782108889
Generation 11: Best fitness = 64.72957013724366
Generation 12: Best fitness = 57.341110436586405
Generation 13: Best fitness = 56.086700303746824
Generation 14: Best fitness = 49.042330144908426
Generation 15: Best fitness = 49.83199836140758
Generation 16: Best fitness = 36.05854667001407
Generation 17: Best fitness = 30.51893901072268
Generation 18: Best fitness = 13.044237011950024
Generation 19: Best fitness = 4.669050337508191
Generation 20: Best fitness = 1.98717101130584

In [95]:
llm_evo = LLMEvolutionStrategy(
            api_key=api_key,
            model="gpt-4o",
            system_prompt_template=system_prompt_template,
            user_prompt_template=user_prompt_template,
            dimensions=dimensions,
            population_size=population_size,
            top_k=4
        )

x_t = torch.rand(population_size, *dimensions).numpy()
fscore_list = [-schwefel_function(genome) for genome in x_t]

max_score = -np.inf
xbest=[]
max_generation=30
for idx in range(max_generation):
    x_t = llm_evo.mutate(x_t, fscore_list, sigma_low=0.05, 
                        sigma_high=0.2, max_retries=3)
    fscore_list = [-schwefel_function(genome) for genome in x_t]
    max_index, xbest_cost = max(enumerate(fscore_list), key=lambda x: x[1])
    if max_score < xbest_cost:
        max_score=xbest_cost
        xbest = x_t[max_index]
    print(f"Generation {idx}: Best fitness = {-xbest_cost}")

Generation 0: Best fitness = 51.09368433070975
Generation 1: Best fitness = 46.912671432092104
Generation 2: Best fitness = 41.934308196521194
Generation 3: Best fitness = 41.10941719886552
Generation 4: Best fitness = 40.40907769418375
Generation 5: Best fitness = 38.30297279292846
Generation 6: Best fitness = 40.59907875185178
Generation 7: Best fitness = 42.7608268091887
Generation 8: Best fitness = 46.866526261556764
Generation 9: Best fitness = 67.33734021550917
Generation 10: Best fitness = 51.431980405697985
Generation 11: Best fitness = 32.196446758613234
Generation 12: Best fitness = 28.13921783198402
Generation 13: Best fitness = 28.429364704639625
Generation 14: Best fitness = 40.868724035290256
Generation 15: Best fitness = 45.759377906449444
Generation 16: Best fitness = 60.08684351508931
Generation 17: Best fitness = 46.09152420975981
Generation 18: Best fitness = 39.720798883329
Generation 19: Best fitness = 43.33814536594765
Generation 20: Best fitness = 45.721349223613